In [49]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf

import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns

import requests
import csv

import warnings

In [50]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

In [51]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [52]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

In [53]:
# Pivot the DataFrame
pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
pivot_sector_ticker_df = pivot_sector_ticker_df.set_index('Sector').T

pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBWI, BBY, BKNG, BWA, ...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, ACGL, AJG, AIZ,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, BIO, TECH...","[MMM, AOS, ALLE, AAL, AME, ADP, AXON, BA, BR, ...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [54]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# Define the ticker symbols as a list; eg. TSM,MSFT,AMZN
# ticker_symbols = input("Enter stock tickers separated by commas:") 
# ticker_symbols = ticker_symbols.split(',')
ticker_symbols = sector_ticker_list

# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 6 # this will return x-1 yr PE range

ticker_dict = {}

In [57]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()

sector_overview_df = pd.DataFrame(columns = data.keys())

for symbol in sector_ticker_df['Ticker']:
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    temp_data_df = pd.DataFrame(data.items()).transpose()

    # Check if DataFrame is not empty and has more than one row        
    if not temp_data_df.empty and len(temp_data_df) > 1:
        temp_data_df.columns = temp_data_df.iloc[0]
        temp_data_df = temp_data_df[1::]
        
        sector_overview_df = pd.concat([sector_overview_df, temp_data_df], ignore_index=True)

    else:
        print(f"{symbol} DataFrame is empty or does not have enough rows to set the first row as header.")

BRK.B DataFrame is empty or does not have enough rows to set the first row as header.
BF.B DataFrame is empty or does not have enough rows to set the first row as header.


In [59]:
sector_overview_df.to_excel('sp500.xlsx')

In [59]:
# Daily quote section
for symbol in ticker_symbols:


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':

            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': f'{symbol}'
                    }
                ,inplace=True
                )
            
            Daily_stock_df[f'{symbol}'] = Daily_stock_df[f'{symbol}'].astype(str).apply(lambda x: float(x))
            Daily_stock_df[f'{symbol}'] = Daily_stock_df[f'{symbol}'].round(2)


    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': f'{symbol}'
            }
        ,inplace=True
        )

    Monthly_stock_df[f'{symbol}'] = Monthly_stock_df[f'{symbol}'].astype(str).apply(lambda x: float(x))
    Monthly_stock_df[f'{symbol}'] = Monthly_stock_df[f'{symbol}'].round(2)



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - 6) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue

            # annualEPS_df[f'{symbol}_PE'] = Monthly_stock_df[f'{symbol}'].values / annualEPS_df['reportedEPS'].values
            # annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"{symbol}_PE"].mean().round(2)
            # annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df[f"{symbol}_PE"]).round(2)
            # annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
            # annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[f'{col}'] = qtrEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API
    # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        forecast_earanings_df = pd.DataFrame(
            columns=my_list[0]
            ,data=my_list[1::]
            )
        
        if forecast_earanings_df['estimate'].head(1).values != '':
            latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
        else:
            latest_projected_EPS = 0


    # forecast 1 year earnings from nasdaq webscrapping




    # Consolidated section
    df_stock_consolidate = Daily_stock_df.head(window_days)


    df_stock_consolidate_date = df_stock_consolidate.index
    for i in df_stock_consolidate_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate']<= i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        df_stock_consolidate.loc[i, f"{symbol}_EPS_TTM"] = EPS_TTM

        if i == max(df_stock_consolidate.index):
            EPS_latest_projected = latest_projected_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        df_stock_consolidate[f"{symbol}_EPS_latest_projected"] = EPS_latest_projected
    
    # stock's stats
    df_stock_consolidate[f"{symbol}_PE_TTM"] = (df_stock_consolidate[symbol] / df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_avg"] = df_stock_consolidate[f"{symbol}_PE_TTM"].mean().round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_std"] = np.std(df_stock_consolidate[f"{symbol}_PE_TTM"]).round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_avg"] + df_stock_consolidate[f"{symbol}_PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_avg"] - df_stock_consolidate[f"{symbol}_PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    df_stock_consolidate[f"{symbol}_relative_valuation_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] * df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    df_stock_consolidate[f"{symbol}_relative_valuation_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] * df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    df_stock_consolidate[f"{symbol}_relative_valuation_median"] = (np.median([df_stock_consolidate[f"{symbol}_relative_valuation_+"], df_stock_consolidate[f"{symbol}_relative_valuation_-"]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    df_stock_consolidate[f"{symbol}_relative_valuation_projected_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] * df_stock_consolidate[f"{symbol}_EPS_latest_projected"]).round(2) # 这个是relative valuation的价格上限
    df_stock_consolidate[f"{symbol}_relative_valuation_projected_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] * df_stock_consolidate[f"{symbol}_EPS_latest_projected"]).round(2) # 这个是relative valuation的价格下限
    df_stock_consolidate[f"{symbol}_relative_valuation_projected_median"] = (np.median([df_stock_consolidate[f"{symbol}_relative_valuation_projected_+"], df_stock_consolidate[f"{symbol}_relative_valuation_projected_-"]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    df_stock_consolidate[f"{symbol}_{window_days}_price_min"] = df_stock_consolidate[symbol].min().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_max"] = df_stock_consolidate[symbol].max().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_avg"] = df_stock_consolidate[symbol].mean().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_std"] = np.std(df_stock_consolidate[symbol]).round(2)

    # df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"].values[0]
    # df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"].values[0]
    # df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    # df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    conditions = [
    (df_stock_consolidate[f"{symbol}"] < df_stock_consolidate[f"{symbol}_relative_valuation_-"]),
    (df_stock_consolidate[f"{symbol}"] > df_stock_consolidate[f"{symbol}_relative_valuation_+"]),
    ((df_stock_consolidate[f"{symbol}"] >= df_stock_consolidate[f"{symbol}_relative_valuation_-"]) & (df_stock_consolidate[f"{symbol}"] <= df_stock_consolidate[f"{symbol}_relative_valuation_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    df_stock_consolidate[f"{symbol}_curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        df_stock_consolidate.loc[condition, f"{symbol}_price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    f"{symbol}"
    # ,f"{symbol}_EPS_TTM"
    # ,f"{symbol}_EPS_latest_projected"
    ,f"{symbol}_PE_TTM"
    ,f"{symbol}_PE_TTM_avg"
    # ,f"{symbol}_PE_{PE_yr_range-1}yr_avg"
    # ,f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"
    # ,f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"
    ,f"{symbol}_relative_valuation_+"
    ,f"{symbol}_relative_valuation_-"
    ,f"{symbol}_relative_valuation_median"
    ,f"{symbol}_relative_valuation_projected_+"
    ,f"{symbol}_relative_valuation_projected_-"
    ,f"{symbol}_relative_valuation_projected_median"
    ,f"{symbol}_price_valuation_assessment"
    ]

    ticker_dict[f'{symbol}'] = df_stock_consolidate[selected_cols]

    # Addtional setting session
    # Set display options to show all rows and columns
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    warnings.filterwarnings("ignore")
    pd.options.mode.copy_on_write = True

In [61]:
ticker_dict.keys()

dict_keys(['MO', 'ADM', 'BG', 'CPB', 'CHD', 'CLX', 'KO', 'CL', 'CAG', 'STZ', 'COST', 'DG', 'DLTR', 'EL', 'GIS', 'HSY', 'HRL', 'K', 'KVUE', 'KDP', 'KMB', 'KHC', 'KR', 'LW', 'MKC', 'TAP', 'MDLZ', 'MNST', 'PEP', 'PM', 'PG', 'SJM', 'SYY', 'TGT', 'TSN', 'WBA', 'WMT'])

In [62]:
ticker_dict['COST']

,COST,COST_PE_TTM,COST_PE_TTM_avg,COST_relative_valuation_+,COST_relative_valuation_-,COST_relative_valuation_median,COST_relative_valuation_projected_+,COST_relative_valuation_projected_-,COST_relative_valuation_projected_median,COST_price_valuation_assessment
2024-05-24,809.73,56.62,51.29,763.33,703.56,733.44,800.17,737.51,768.84,overvalued
2024-05-23,796.34,55.69,51.29,763.33,703.56,733.44,800.17,737.51,768.84,overvalued
2024-05-22,801.86,56.07,51.29,763.33,703.56,733.44,800.17,737.51,768.84,overvalued
2024-05-21,800.93,56.01,51.29,763.33,703.56,733.44,800.17,737.51,768.84,overvalued
2024-05-20,793.00,55.45,51.29,763.33,703.56,733.44,800.17,737.51,768.84,overvalued
2024-05-17,795.81,55.65,51.29,763.33,703.56,733.44,800.17,737.51,768.84,overvalued
2024-05-16,793.07,55.46,51.29,763.33,703.56,733.44,800.17,737.51,768.84,overvalued
2024-05-15,787.04,55.04,51.29,763.33,703.56,733.44,800.17,737.51,768.84,overvalued
2024-05-14,777.90,54.40,51.29,763.33,703.56,733.44,800.17,737.51,768.84,overvalued
2024-05-13,775.15,54.21,51.29,763.33,703.56,733.44,800.17,737.51,768.84,overvalued


# IPO

In [138]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=IPO_CALENDAR&apikey={alpha_vantage_api_key}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

    IPO_df = pd.DataFrame(columns=my_list[0]
                          ,data=my_list[1::])

In [139]:
IPO_df

,symbol,name,ipoDate,priceRangeLow,priceRangeHigh,currency,exchange
0,I,n,f,o,r,m,a
